In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [2]:
import numpy as np
import datetime
from datetime import datetime

In [ ]:
driver = webdriver.Safari(executable_path ='/usr/bin/safaridriver')
df = pd.DataFrame(columns=['time', 'temp', 'dew_point', 'humidity', 'wind', 'wind_speed', 'wind_gust', 'pressure', 'precip', 'condition'])

for i in range(1,13):
    days_in_month = 0
    if i in [1,3,5,7,8,10,12]:
        days_in_month = 32
    elif i in [4,6,9,11]:
        days_in_month = 31
    elif i in [2]:
        days_in_month = 29
    for j in range(1,days_in_month):
        date = f'{j}.{i}.2017'
        print(date)
        url = f'https://www.wunderground.com/history/daily/KMDW/date/2017-{i}-{j}'
        driver.get(url)
        time.sleep(5)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find('table', attrs={'class': 'mat-table cdk-table mat-sort ng-star-inserted' })
        table_rows = table.find_all('tr')
        l = []
        for tr in table_rows:       
            td = tr.find_all('td')
            row = [tr.text for tr in td]
            l.append(row)
            l = list(filter(None, l))
        for k in l:
            df.loc[len(df)] = k
            

driver.quit()
      


In [50]:
df.tail()

,time,temp,dew_point,humidity,wind,wind_speed,wind_gust,pressure,precip,condition
10245,8:53 PM,6 °F,-5 °F,60 °%,NW,10 °mph,20 °mph,29.92 °in,0.0 °in,Fair
10246,9:53 PM,4 °F,-6 °F,64 °%,NW,10 °mph,0 °mph,29.92 °in,0.0 °in,Fair
10247,10:53 PM,4 °F,-8 °F,58 °%,WNW,12 °mph,0 °mph,29.94 °in,0.0 °in,Fair
10248,11:53 PM,2 °F,-8 °F,63 °%,WNW,7 °mph,0 °mph,29.95 °in,0.0 °in,Fair
10249,12:53 AM,1 °F,-11 °F,57 °%,WNW,12 °mph,0 °mph,29.95 °in,0.0 °in,Fair


In [51]:
#save scraped data in csv
df.to_csv('./data/weather_data.csv', index = False)

In [17]:
#load scraped data for further processing
df = pd.read_csv('./data/weather_data.csv')

In [18]:
#add date column
df['date'] = np.nan

In [22]:
#filter out all datapoints that have been collected outside of the hourly (XX:53 AM/PM) weather measurements 
tuple = ('53 AM', "53 PM")
df_hourly_measurement = df[df['time'].str.endswith(tuple)]
len(df_hourly_measurement)

8749

PROBLEM: 11 Values are missing, see below.

In [31]:
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.startswith('1:53 AM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.startswith('2:53 AM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.endswith('3:53 AM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.endswith('4:53 AM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.endswith('5:53 AM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.endswith('6:53 AM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.endswith('7:53 AM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.endswith('8:53 AM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.endswith('9:53 AM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.endswith('10:53 AM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.endswith('11:53 AM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.endswith('12:53 AM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.startswith('1:53 PM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.startswith('2:53 PM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.endswith('3:53 PM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.endswith('4:53 PM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.endswith('5:53 PM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.endswith('6:53 PM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.endswith('7:53 PM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.endswith('8:53 PM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.endswith('9:53 PM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.endswith('10:53 PM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.endswith('11:53 PM')]))
print(len(df_hourly_measurement[df_hourly_measurement['time'].str.endswith('12:53 PM')]))

364
364
364
363
365
365
365
365
365
365
365
364
364
365
365
365
365
365
366
363
365
364
363
365


In [33]:
df_hourly_measurement = df_hourly_measurement.reset_index()

In [ ]:
# add dates to data measurements
first_hour = 0
last_hour = 24
day = 1

for i in range(1,13):
    days_in_month = 0
    if i in [1,3,5,7,8,10,12]:
        days_in_month = 32
    elif i in [4,6,9,11]:
        days_in_month = 31
    elif i in [2]:
        days_in_month = 29
    for j in range(1,days_in_month):
        r = range(first_hour,last_hour)
        df_hourly_measurement.loc[r,['date']] = [f'2017-{i}-{j}']
        first_hour = first_hour + 24
        last_hour = last_hour + 24

df_hourly_measurement.head()



In [146]:
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')